In [1]:
import os
import ee
import geemap
import time
import random
from datetime import datetime, timedelta , date
import ipywidgets as widgets
from IPython.display import display
from tkinter import Tk, filedialog
import traitlets
from ipyfilechooser import FileChooser

In [2]:
class SelectFilesButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""

    def __init__(self):
        super(SelectFilesButton, self).__init__()
        # Add the selected_files trait
        self.add_traits(files=traitlets.traitlets.List())
        # Create the button.
        self.description = "Select Files"
        self.icon = "square-o"
        self.style.button_color = "orange"
        # Set on click behavior.
        self.on_click(self.select_files)

    @staticmethod
    def select_files(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askopenfilename(multiple=True,filetypes=[('shp','.shp')])

        b.description = "Files Selected"
        b.icon = "check-square-o"
        b.style.button_color = "lightgreen"

In [4]:
# After executing this line of code for the first use, you can get the authentication number linked to Google.
#Map = geemap.Map()
# Authenticate the Google earth engine with google account
ee.Initialize() 
Map = geemap.Map()

In [5]:
# give the shp file
my_button = SelectFilesButton()
my_button # This will display the button in the context of Jupyter Notebook

SelectFilesButton(description='Select Files', icon='square-o', style=ButtonStyle(button_color='orange'))

In [6]:
# give the star date and end date

star = widgets.DatePicker(
    description='Pick a Star Date',
    disabled=False
)
end = widgets.DatePicker(
    description='Pick a End Date',
    disabled=False
)

widgets.HBox([star, end])

In [7]:
# give the bands

band_name = widgets.Dropdown(
options=['ndvi','evi','savi','ndwi1','ndwi2','ndwi3'],
description='Band')

band_name

Dropdown(description='Band', options=('ndvi', 'evi', 'savi'), value='ndvi')

In [8]:
# gine the date frequency

frequency =widgets.Dropdown(
    options=['Year','Month'],
    value='Year',
    description='Frequency')

frequency

Dropdown(description='Frequency', options=('Year', 'Month'), value='Year')

In [9]:
# give the output floder and flie name
folder = FileChooser()
display(folder)


FileChooser(path='C:\Users\Yang Hsiu\Desktop\satellite python code\satellite python code\satellite python code…

In [10]:
# give the scale number
scale = widgets.IntSlider(
    value=1000,
    min=500,
    max=10000,
    step=100,
    description='Scale:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

scale

IntSlider(value=1000, continuous_update=False, description='Scale:', max=10000, min=500, step=100)

In [19]:
def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + timedelta(days=4)  # this will never fail
    return next_month - timedelta(days=next_month.day)

def monthlist(begin,end):
    #begin = datetime.datetime.strptime(begin, "%Y-%m-%d")
    #end = datetime.datetime.strptime(end, "%Y-%m-%d")

    result = []
    while True:
        if begin.month == 12:
            next_month = begin.replace(year=begin.year+1,month=1, day=1)
        else:
            next_month = begin.replace(month=begin.month+1, day=1)
        if next_month > end:
            break
        result.append ([begin.strftime("%Y-%m-%d"),last_day_of_month(begin).strftime("%Y-%m-%d")])
        begin = next_month
    result.append ([begin.strftime("%Y-%m-%d"),end.strftime("%Y-%m-%d")])
    return result


def last_day_of_year(any_day):
    next_year = any_day.replace(year=any_day.year, month=12, day = 31)  # this will never fail
    return next_year

def yearlist(begin,end):
    #begin = datetime.datetime.strptime(begin, "%Y-%m-%d")
    #end = datetime.datetime.strptime(end, "%Y-%m-%d")

    result = []
    while True:
        if begin.year < end.year:
            next_year = begin.replace(year=begin.year+1,month=1, day=1)
        else:
            next_year = end
        if next_year == end:
            break
        result.append ([begin.strftime("%Y-%m-%d"),last_day_of_year(begin).strftime("%Y-%m-%d")])
        begin = next_year
    result.append ([begin.strftime("%Y-%m-%d"),end.strftime("%Y-%m-%d")])
    return result

In [20]:
enddate = (end.value + timedelta(days=1)).strftime("%Y-%m-%d")
stardate = star.value.strftime("%Y-%m-%d")
roi = geemap.shp_to_ee(''.join(my_button.files)).geometry()
dateFilter = ee.Filter.date(str(stardate), str(enddate))
#Load a MODIS collection with data.
mcd43a4 = ee.ImageCollection("MODIS/006/MCD43A4")\
            .select(['Nadir_Reflectance_Band1','Nadir_Reflectance_Band2','Nadir_Reflectance_Band3',
            'Nadir_Reflectance_Band4','Nadir_Reflectance_Band5','Nadir_Reflectance_Band6',
            'Nadir_Reflectance_Band7'],
            ['red', 'nir', 'blue', 'green', 'swir1', 'swir2', 'swir3']);


#Load a MODIS collection with quality data.
mcd43a2 = ee.ImageCollection('MODIS/006/MCD43A2')\
            .select(['BRDF_Albedo_Band_Quality_Band1', 'BRDF_Albedo_Band_Quality_Band2', 'BRDF_Albedo_Band_Quality_Band3',
            'BRDF_Albedo_Band_Quality_Band4', 'BRDF_Albedo_Band_Quality_Band5', 'BRDF_Albedo_Band_Quality_Band6',
            'BRDF_Albedo_Band_Quality_Band7', 'BRDF_Albedo_LandWaterType'],
            ['qa1', 'qa2', 'qa3', 'qa4', 'qa5', 'qa6', 'qa7', 'water']);


    
    
    #Define an inner join.
innerJoin = ee.Join.inner('NBAR', 'QA');

#Specify an equals filter for image timestamps.
filterTimeEq = ee.Filter.equals(
  leftField ='system:time_start',
  rightField ='system:time_start'
);

#Apply the join.
innerJoinedMODIS = innerJoin.apply(mcd43a4, mcd43a2, filterTimeEq);

In [21]:
# get indexes NDVI
def getNDVI(image):
    
    # Normalized difference vegetation index (NDVI)
    ndvi = image.normalizedDifference(['nir','red']).rename("ndvi")
    image = image.addBands(ndvi)

    return(image)

def getEVI(image):
    evi = image.expression(
      '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR' : image.select('nir'),
      'RED' : image.select('red'),
      'BLUE': image.select('blue')}).rename('evi');
    return image.addBands(evi);
    

def getSAVI(image):
    savi = image.expression(
     '1.5 * (nir - red) / (nir + red + 0.5)', {
      'nir': image.select('nir'),
      'red': image.select('red')}).rename('savi');
    return image.addBands(savi);

# get indexes NDWI1
def getNDWI1(image):
    
    # Normalized difference vegetation index (NDWI1) (Xnir - Xswir1)/(Xnir + Xswir1)
    ndwi1 = image.normalizedDifference(['nir','swir1']).rename("ndwi1")
    image = image.addBands(ndwi1)

    return(image)

# get indexes NDWI2
def getNDWI2(image):
    
    # Normalized difference vegetation index (NDWI2) (Xnir - Xswir2)/(Xnir + Xswir2)
    ndwi2 = image.normalizedDifference(['nir','swir2']).rename("ndwi2")
    image = image.addBands(ndwi2)

    return(image)

# get indexes NDWI3
def getNDWI3(image):
    
    # Normalized difference vegetation index (NDWI3) (Xgreen - Xnir)/(Xgreen + Xnir)
    ndwi3 = image.normalizedDifference(['green','nir']).rename("ndwi3")
    image = image.addBands(ndwi3)

    return(image)


def addQABands(image):
    nbar = ee.Image(image.get('NBAR'));
    qa = ee.Image(image.get('QA')).select(['qa2']);
    water = ee.Image(image.get('QA')).select(['water']);
    return(nbar.addBands([qa, water]));

def Modis_filter(image):
    qaband = image.select(['qa2']); #Right now, only using QA info for the NIR band
    wband = image.select(['water']);
    qamask = qaband.lte(2) and (wband.eq(1));
    nir_r = image.select('nir').multiply(0.0001).rename('nir');
    red_r = image.select('red').multiply(0.0001).rename('red');
    swir1_r = image.select('swir1').multiply(0.0001).rename('swir1');
    swir2_r = image.select('swir2').multiply(0.0001).rename('swir2');
    blue_r = image.select('blue').multiply(0.0001).rename('blue');
    return image.addBands(nir_r).addBands(red_r).addBands(swir1_r).addBands(swir2_r).addBands(blue_r).updateMask(qamask)


In [22]:
#Merge two layers
mergedCol = ee.ImageCollection(innerJoinedMODIS.map(addQABands))
Modis_filter_vars = mergedCol.map(Modis_filter)

In [23]:
def geotif(time_list):
   
    for i in range(0,len(time_list)):
        star_time = time.time()
    
        dataset = ee.ImageCollection(Modis_filter_vars) \
                    .filterDate(time_list[i][0],datetime.strptime(time_list[i][1],"%Y-%m-%d")+ timedelta(days=1)) \
                    .filterBounds(roi) \
                    .map(getNDVI) \
                    .map(getEVI) \
                    .map(getSAVI) \
                    .map(getNDWI1) \
                    .map(getNDWI2) \
                    .map(getNDWI3) \
                    .select(band_name.value) \
                    .mean()
    
        image = dataset.clip(roi).unmask()
        filename = os.path.join(os.path.expanduser(folder.selected) + '_{}.tif'.format(time_list[i]))
        geemap.ee_export_image(
            image, filename=filename, scale=scale.value, region=roi, file_per_band=False
                                )
    
    

In [ ]:
if 'Year' in frequency.value.strip():
    time_list = yearlist(star.value, end.value)
    geotif(time_list)
else:
    pass
if "Month" in frequency.value.strip():
    time_list =monthlist(star.value, end.value)
    geotif(time_list)
else:
    pass

Generating URL ...
Please wait ...
